---
Eli Schwat

elilouis@uw.edu

Created for Professor Michael Brett's CEWA547 Course, Winter 2021

---

# Analyze Salish Sea Model Point Source Inputs

This notebook walks you through an analysis of non-river point sources to to the Salish Sea model.

User input is required when you see this...

**<span style="color:red">USER INPUT REQUIRED</span>**

In [1]:
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

# Input Variables:

**<span style="color:red">USER INPUT REQUIRED</span>**

Put in the path to the file `ssm_pnt_wq.dat` (or similar) that should come with your packaging of the SSM model.

In [2]:
# input_file = "/Users/elischwat/Google Drive/UW/Classes Winter 2021/Watershed MGMT/salish sea model/SSM_WQM_model_inputs/inputs/ssm_pnt_wq.dat"
# input_file = "/Users/elischwat/Google Drive/UW/Classes Winter 2021/Watershed MGMT/salish sea model/SSM_WQM_model_inputs/inputs/ssm_pnt_wqMODIFIED.dat"
input_file = "/Users/elischwat/Downloads/inputs_modifie/inputs/ssm_pnt_wqMODIFIED0.dat"

In [3]:
!cat $input_file | head -50

point  calculated !2014
193
   8778 ! FVCOM ID/Node: 87 / 3856 [Distributed: YES], Fraser - River (ECY ID: 258) --- Vertical Distriubtion: Power law, Notes: No change
   8861 ! FVCOM ID/Node: 88 / 3919 [Distributed: YES], Fraser - River (ECY ID: 258) --- Vertical Distriubtion: Power law, Notes: No change
   7542 ! FVCOM ID/Node: 59 / 2914 [Distributed: YES], Nooksack - River (ECY ID: 238) --- Vertical Distriubtion: Power law, Notes: No change
   7543 ! FVCOM ID/Node: 60 / 2915 [Distributed: YES], Nooksack - River (ECY ID: 238) --- Vertical Distriubtion: Power law, Notes: No change
   7015 ! FVCOM ID/Node: 69 / 2543 [Distributed: YES], Samish_Bell south - River (ECY ID: 246) --- Vertical Distriubtion: Power law, Notes: No change
   7016 ! FVCOM ID/Node: 70 / 2544 [Distributed: YES], Samish_Bell south - River (ECY ID: 246) --- Vertical Distriubtion: Power law, Notes: No change
  12754 ! FVCOM ID/Node: 73 / 6350 [Distributed: YES], Skagit - River (ECY ID: 249) --- Vertical Distriubtion: P

In [4]:
variable_name_dict = {
    0: "Flow (OFF, b/c from FVCOM)",
    1: "Temperature (OFF, b/c from FVCOM)",
    2: "Salinity (OFF, b/c from FVCOM)",
    3: "TSS",
    4: "Algal 1 (Algal group 1)",
    5: "Algal 2 (Algal group 2)",
    6: "Algal 3 (Algal group 3) (unused)",
    7: "Zooplankton 1 (Zooplankton – species 1)",
    8: "Zooplankton 2 (Zooplankton species 2)",
    9: "Labile DOC (Labile dissolved organic carbon)",
    10: "Refractory DOC (Refractory dissolved organic carbon)",
    11: "Labile POC (Labile particulate organic carbon)",
    12: "Refractory POC (Refractory particulate organic carbon)",
    13: "Ammonium (NH4)",
    14: "Nitrate + Nitrite (NO3+NO2)",
    15: "Urea",
    16: "Labile DON (Labile dissolved organic nitrogen)",
    17: "Refractory DON (Refractory dissolved organic nitrogen)",
    18: "Labile PON (Labile particular organic nitrogen)",
    19: "Refractory PON (Refractory particulate organic nitrogen)",
    20: "Total PO4 (Total phosphate)",
    21: "Labile DOP (Labile dissolved organic phosphate)",
    22: "Refractory DOP (Refractory dissolved organic phosphate)",
    23: "Labile POP (Labile particulate organic phosphate)",
    24: "Refractory POP (Refractory particulate organic phosphate)",
    25: "Particulate inorganic P (Particulate inorganic phosphate)",
    26: "COD (Chemical oxygen demand)",
    27: "DO (Dissolved Oxygen)",
    28: "Particulate Silica",
    29: "Dissolved Silica",
    30: "internal P group for Alga 1, Droop model (currently off)",
    31: "internal P group for Alga 2, Droop model (currently off)",
    32: "internal P group for Alga 3, Droop model (currently off)",
    33: "DIC",
    34: "Alkalinity"
}

In [5]:
def read_data(input_file, num_params):
    """
    Params:
    input_file (str): path to input file
    num_params (int): number of parameters contained in the file. Usually 35.
    
    Returns:
    (df, header_lines): df is a dataframe containing the data separated by parameter, point source, 
        and date. header_lines is a list of strings containing all the header data that must be 
        written to the new file.
    """
    with open(input_file) as f:
        lines = [line.rstrip() for line in f]
    num_point_sources = int(lines[1])
    print(f"Found {num_point_sources} point sources")
    header_lines = lines[:num_point_sources*2+3]
    data_lines =  lines[num_point_sources*2+3:]
    num_daily_data = int(header_lines[-1])
    print(f"Found {num_daily_data} days of data")
    df_list = []
    for n_day in range(0, num_daily_data):
        day_num = data_lines[n_day*(num_params+1)]
        lines = data_lines[n_day*(num_params+1) + 1: n_day*(num_params+1) + 1 + num_params]
        df_list.append(__extract_daily_data(lines, day_num))
    df = pd.concat(df_list)
    df = df.reset_index(drop=True)
    df['hour'] = df['hour'].astype('float')
    return df, header_lines
          
def write_data(output_file, df, header_lines):
    """
    Params:
    output_file (str): path to output file.
    df (pandas.DataFrame): a dataframe containing data. such as is returned by the read_data function
                defined above.
    header_lines: list of strings, such as is returned by the read_data function defined above.
    """
    writer = open(output_file, "w")
    writer.write("\n".join(header_lines))
    for day, day_df in df.groupby('hour'):
        #generate a days worth of data which is composed of:
        #1. a first single line with the julian day
        writer.write("\n")
        day_line_string = "     {:.2f}".format(day)
        writer.write(day_line_string)
        print(f"Writing data for day: {day_line_string}")
        #2. num_params lines of data, each line is a series of single-space-separated floats (formatted in sci notation),
        #    each line is num_point_sources floats long. make sure lines are written in the order of the parameter number.    
        for index, row in day_df.iloc[:,2:].iterrows():
            line_string = ' ' + ' '.join([ #add a space here because that's how the original file is
                '{:.3E}'.format(single_param_vals) for single_param_vals in row
            ])
            writer.write("\n")
            writer.write(line_string)
    writer.write("\n") #to put an empty line at the beginning, as the original files have
    writer.close()
          
def __extract_daily_data(lines, day_num):
    assert len(lines)==num_params, f"Expecting {num_params} lines of data"
    arr_list = []
    for i in range(0, len(lines)):
        line = lines[i]
        param_index = i
        arr_list.append(
            [float(x) for x in line.strip().split(' ')]
        )
    df = pd.DataFrame(arr_list)
    df.insert(0, 'hour', day_num)
    df.insert(0, 'param', df.index)
    return df

## Read Data 

In [6]:
num_params = 35

In [7]:
df, header_lines = read_data(input_file, num_params)
source_lines = header_lines[2:195]
source_names_series = pd.Series(source_lines).apply(
    lambda x: 
    x.split(',')[1].split('---')[0].strip() + ' ' + x.split()[0]
)
point_source_types = source_names_series.apply(lambda x: x.split(' - ')[1].split(' (')[0].strip())

Found 193 point sources
Found 365 days of data


In [8]:
len(source_names_series), len(point_source_types)

(193, 193)

## Create human-usable dataframe

In [9]:
hr_df = df.copy().rename(mapper=source_names_series.to_dict(), axis=1)

In [10]:
hr_df

,param,hour,Fraser - River (ECY ID: 258) 8778,Fraser - River (ECY ID: 258) 8861,Nooksack - River (ECY ID: 238) 7542,Nooksack - River (ECY ID: 238) 7543,Samish_Bell south - River (ECY ID: 246) 7015,Samish_Bell south - River (ECY ID: 246) 7016,Skagit - River (ECY ID: 249) 12754,Skagit - River (ECY ID: 249) 12755,...,Macaulay - Point Source (ECY ID: 293) 3144,Saanich - Point Source (ECY ID: 294) 3799,Clallam DOC - Point Source (ECY ID: 295) 1853,Larrabee State Park - Point Source (ECY ID: 296) 6361,Warm Beach Campground - Point Source (ECY ID: 297) 11243,Whidbey Naval Station - Point Source (ECY ID: 298) 4710,Puyallup - Point Source (ECY ID: 299) 13754,Brightwater - Point Source (ECY ID: 300) 8741,Lake Stevens 002 - Point Source (ECY ID: 301) 14229,Messenger House - Point Source (ECY ID: 302) 9571
0,0,0.0,640.00,640.00,31.070,31.070,5.635,5.635,158.700,158.700,...,0.5063,0.113,0.006036,0.000063,0.000985,0.01353,0.1927,0.7577,0.1218,0.00019
1,1,0.0,3.73,3.73,5.939,5.939,5.939,5.939,5.939,5.939,...,15.0600,15.060,15.060000,15.060000,15.060000,15.06000,15.0600,15.0600,15.0600,15.06000
2,2,0.0,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0000,0.000,0.000000,0.000000,0.000000,0.00000,0.0000,0.0000,0.0000,0.00000
3,3,0.0,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0000,0.000,0.000000,0.000000,0.000000,0.00000,0.0000,0.0000,0.0000,0.00000
4,4,0.0,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0000,0.000,0.000000,0.000000,0.000000,0.00000,0.0000,0.0000,0.0000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12770,30,8736.0,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0000,0.000,0.000000,0.000000,0.000000,0.00000,0.0000,0.0000,0.0000,0.00000
12771,31,8736.0,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0000,0.000,0.000000,0.000000,0.000000,0.00000,0.0000,0.0000,0.0000,0.00000
12772,32,8736.0,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0000,0.000,0.000000,0.000000,0.000000,0.00000,0.0000,0.0000,0.0000,0.00000
12773,33,8736.0,1138.00,1138.00,526.800,526.800,601.800,601.800,534.400,534.400,...,3866.0000,3866.000,3866.000000,3866.000000,3866.000000,3866.00000,3866.0000,3866.0000,3866.0000,3866.00000


In [11]:
hr_df = pd.melt(hr_df, id_vars=['hour', 'param']).rename({'variable': 'source'}, axis=1)
params_of_interest = [0,1,2,13,14,15,16,17,18,19]
hr_df = hr_df[hr_df.param.isin(params_of_interest)]
hr_df.param = hr_df.param.apply(lambda x: variable_name_dict.get(x).split('(')[0].strip())

In [12]:
hr_df

,hour,param,source,value
0,0.0,Flow,Fraser - River (ECY ID: 258) 8778,640.0000
1,0.0,Temperature,Fraser - River (ECY ID: 258) 8778,3.7300
2,0.0,Salinity,Fraser - River (ECY ID: 258) 8778,0.0000
13,0.0,Ammonium,Fraser - River (ECY ID: 258) 8778,0.0010
14,0.0,Nitrate + Nitrite,Fraser - River (ECY ID: 258) 8778,0.1161
...,...,...,...,...
2465555,8736.0,Urea,Messenger House - Point Source (ECY ID: 302) 9571,0.0000
2465556,8736.0,Labile DON,Messenger House - Point Source (ECY ID: 302) 9571,0.6157
2465557,8736.0,Refractory DON,Messenger House - Point Source (ECY ID: 302) 9571,0.0000
2465558,8736.0,Labile PON,Messenger House - Point Source (ECY ID: 302) 9571,0.1418


In [13]:
len(hr_df.source.unique())

193

In [14]:
hr_df['source_name'] = hr_df.source.apply(lambda x: x.split('-')[0].strip())
hr_df['type'] = hr_df.source.apply(lambda x: x.split(' - ')[1].split('(')[0].strip())
hr_df['ECY ID'] = hr_df.source.apply(lambda x: x.split(' - ')[1].split('(')[1].split(':')[1].split(')')[0].strip())
hr_df['node'] =  hr_df.source.apply(lambda x: x.split()[-1])

In [15]:
hr_df.head(3)

,hour,param,source,value,source_name,type,ECY ID,node
0,0.0,Flow,Fraser - River (ECY ID: 258) 8778,640.00,Fraser,River,258,8778
1,0.0,Temperature,Fraser - River (ECY ID: 258) 8778,3.73,Fraser,River,258,8778
2,0.0,Salinity,Fraser - River (ECY ID: 258) 8778,0.00,Fraser,River,258,8778


Pivot the table...

In [16]:
hr_df = hr_df.pivot_table(
    index=['hour','source','source_name','type','ECY ID', 'node'], 
    columns='param', 
    values='value'
).rename_axis(None, axis=1).reset_index()

In [17]:
hr_df['day'] = hr_df['hour']/24

In [18]:
hr_df[hr_df.source.str.contains('Fraser')].source.unique()

array(['Fraser - River (ECY ID: 258) 8778',
       'Fraser - River (ECY ID: 258) 8861'], dtype=object)

## Convert Effluent Concentrations to Effluent Mass Fluxes

#### First convert units of the concentration values (mg/L -> mg/m^3)

In [19]:
hr_flux_df = hr_df.copy()

In [20]:
def convert_per_liter_to_per_cubic_meter(mg_per_liter_value):
    cubic_meter_in_1_liter = 0.001
    return mg_per_liter_value * (1 / cubic_meter_in_1_liter)

for col in [
    'Ammonium', 'Nitrate + Nitrite', 'Urea', 'Labile DON', 
    'Refractory DON', 'Labile PON', 'Refractory PON']:
    hr_flux_df[col] = hr_flux_df[col].apply(convert_per_liter_to_per_cubic_meter)

#### Then multiply the concentration values by flow (mg/m^3 * m^3/s = mg/s)

In [21]:
def convert_concentrations_to_mass_flux(df):
    flow_values = df.loc[df.param == 'Flow', 'value']
    assert len(flow_values) == 1, "More than 1 flow value per location and hour...that's a problem"
    flow_value = flow_values.iloc[0]
    df.loc[df.param != 'Flow', 'value'] = df.loc[df.param != 'Flow', 'value'] * flow_value
    return df

In [22]:
for col in [
    'Ammonium', 'Nitrate + Nitrite', 'Urea', 'Labile DON', 
    'Refractory DON', 'Labile PON', 'Refractory PON']:
    hr_flux_df[col] = hr_flux_df[col] * hr_flux_df['Flow']

In [23]:
hr_flux_df.head(3)

,hour,source,source_name,type,ECY ID,node,Ammonium,Flow,Labile DON,Labile PON,Nitrate + Nitrite,Refractory DON,Refractory PON,Salinity,Temperature,Urea,day
0,0.0,Alderbrook - Point Source (ECY ID: 234) 13626,Alderbrook,Point Source,234,13626,3.546004,0.000832,0.389802,0.202308,5.116366,0.0,0.0,0.0,15.06,0.0,0.0
1,0.0,Alderwood - Point Source (ECY ID: 235) 8479,Alderwood,Point Source,235,8479,10218.208000,0.400400,126.446320,49.008960,2743.540800,0.0,0.0,0.0,15.06,0.0,0.0
2,0.0,Anacortes - Point Source (ECY ID: 236) 5716,Anacortes,Point Source,236,5716,2927.144000,0.114700,36.222260,14.039280,785.924400,0.0,0.0,0.0,15.06,0.0,0.0


## Calculate Total Nitrogen Output per Source & Find Top Polluters (total N mass flux (mg N/second))

In [24]:
hr_flux_df['Total Nitrogen'] = hr_flux_df[
    ['Ammonium', 'Nitrate + Nitrite', 'Urea', 'Labile DON', 'Refractory DON', 'Labile PON', 'Refractory PON']
].sum(axis='columns')

Calculate total annual N flux (kg/year)

In [25]:
annual_flux_df = hr_flux_df.groupby(['source', 'source_name', 'type', 'ECY ID', 'node']).mean().reset_index()

In [26]:
annual_flux_df['Total Nitrogen'] = (annual_flux_df['Total Nitrogen'] * 365*24*60*60 * 1e-6).astype('int')

In [27]:
modified_sources = [
    "South King",
    "West Point",
    "Chambers Creek",
    "Brightwater",
    "Tacoma Central",
    "Bellingham",
    "Everett Snohomish",
    "Lakota",
]

In [28]:
annual_flux_aois_df = annual_flux_df[annual_flux_df.source_name.isin(modified_sources)]
annual_flux_aois_df = annual_flux_aois_df[annual_flux_aois_df.type == 'Point Source']

In [29]:
len(annual_flux_aois_df)

8

In [30]:
print(input_file)
print('Total N from modified sources:')
print(annual_flux_aois_df['Total Nitrogen'].sum()/1000)
print('Total N from all non-river point sources:')
print(annual_flux_df[annual_flux_df.type == 'Point Source']['Total Nitrogen'].sum()/1000)
print('Total N from all sources:')
print(annual_flux_df['Total Nitrogen'].sum()/1000)

/Users/elischwat/Downloads/inputs_modifie/inputs/ssm_pnt_wqMODIFIED0.dat
Total N from modified sources:
0.0
Total N from all non-river point sources:
16278.34
Total N from all sources:
64542.166
